## Simple Web Scraping for Collecting Data

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

----

----

In [ ]:
# To run the code without chrome driver. Run this cell and skip the next cell.

# # Make a request to the webpage
# page = requests.get('https://www.imsnsit.org/imsnsit/notifications.php')

# # Create a BeautifulSoup object
# soup = BeautifulSoup(page.content, 'html.parser')

### Click Button with XPATH

XPATH is different for each element. It can be obtained simply from Google Chrome. Go to the website, right click and select inspect. Try to locate the element. Chrome will highlight the element on which your cursor is hovering. This process may take some time. 

Right click the element in the inspect tab and select copy xpath.

https://medium.com/ymedialabs-innovation/web-scraping-using-beautiful-soup-and-selenium-for-dynamic-page-2f8ad15efe25

In [ ]:
#Create a web driver object
driver = webdriver.Chrome('/Users/anantgokhale/webscrape-attempt/chromedriver_mac_arm64 (1)/chromedriver')
driver.implicitly_wait(0.5)
driver.maximize_window()
#Mention url in string that driver needs to fetch
url = 'https://www.imsnsit.org/imsnsit/notifications.php'
driver.get(url)
#Copy the xpath by inspecting the page and paste it here 
button = driver.find_element("xpath", '//*[@id="olddata"]')
#Perform click
button.click()

#Now we will use BeautifulSoup and thus create a soup object
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
# soup.prettify()

In [ ]:
#function to print all elements in a list
def disp_list_2(list):
    for idx, item in enumerate(list):
        print('({}) {}'.format(idx,item))
    

In [ ]:
#function to remove some suffix from a string
def my_removesuffix(s, suffix):
    return s[:-len(suffix)] if s.endswith(suffix) else s
# https://note.nkmk.me/en/python-str-remove-strip/

In [ ]:
import string

#Using string library define a function that replaces any punctuations present in a string with a blankspace 
def replace_punctuation(input_string):
    # Make a translation table
    translator = str.maketrans(string.punctuation, " " * len(string.punctuation))

    # Use this table to replace all punctuation with spaces
    no_punct = input_string.translate(translator)
    return no_punct


In [ ]:
def function(list):
    for idx, each in enumerate(list):
        list[idx] = each.lower().replace('semester','sem')

## Method 1

In [ ]:
all_font_tags = soup.find_all('font')


In [ ]:
#create a copy of all_font_tags
all_font_tags_copy = all_font_tags

#select list items 7th index onwards with a jump of 3
all_font_tags = all_font_tags[7::3]
print(len(all_font_tags))

#Take first 251 elements. 252nd element is not available
all_font_tags = all_font_tags[0:251]
print(len(all_font_tags))



In [ ]:
#Skipping 252 and taking the 253rd element onwards
all_font_tags[252::] = all_font_tags_copy[7+251*3+5::3]
#This works well till ~380 index

PROBLEM OCCURING FROM NOTICE IN: 15-12-2022. NOTICE - Classes for 1 semester backlog courses for 2021 batch admitted B.Tech students only shall commence from 19.12.2022

In [ ]:
published_by_list = soup.find_all('b')
#First 3 b_tags are irrelevant, therefore
published_by_list = published_by_list[3::]



In [ ]:
# Replace Published By: part 
for i in range(len(published_by_list)):
    published_by_list[i] = (published_by_list[i].text).replace('Published By:','')
    # print('({})'.format(i),end ='')
    # print(published_by_list[i])   

In [ ]:
list101 =[]
for each in all_font_tags:
    list101.append(replace_punctuation(each))

list102 =[]
for each in published_by_list:
    list102.append(replace_punctuation(each))    

## Method 2:

In [ ]:
def process(Data):
    list201 = []
    for each in Data:
        list201.append(replace_punctuation(each.text))
    print(len(list201))     

In [ ]:
Data = soup.findAll('td', {'class' : 'list-data-focus', 'colspan' : '2'})

In [ ]:
list201 = []
for each in Data:
    list201.append(replace_punctuation(each.text))
print(len(list201))    

In [ ]:
list201.reverse()

In [ ]:
def perform_request(seconds = 0):

    import time, requests
    # Wait for seconds 
    time.sleep(seconds)
    # Reperform request
    page = requests.get('https://www.imsnsit.org/imsnsit/notifications.php')
    soup = BeautifulSoup(page.content, 'html.parser')
    newData = soup.findAll('td', {'class' : 'list-data-focus', 'colspan' : '2'})
    new_list = process(newData)
    # Check size of list of strings. Hoping no change in length
    if len(new_list) !=50:
        print('The code broke'+
            '(1) The ims Notices and Circulars HTML layout changed'+
            '(2) Different HTML tags are being used to store notices')
        return -1     
    else:
        return new_list  

In [ ]:
def rel_position(new_list):
    if (not new_list):
        return -1
    else:
        new_data_var = 0
        old_data_list = list201[-1:-51] 
        for i in range(1,51):
            if new_list[-1].text == old_data_list[-i]:
                new_data_var = -1 - -i
                break
            if (i==10):
                print('10 or more new notices found')
            if (i==50):
                print('50 or more new notices found, or the code broke'+
                    '(1) The ims Notices and Circulars HTML layout changed'+
                    '(2) Different HTML tags are being used to store notices') 
                return -1
        new_notices=[]        
        for idx,each in enumerate(new_list):
                if idx < new_data_var:
                    new_notices.append (new_list[idx])
        return new_notices     

In [ ]:
new_list = perform_request()
new_notices = rel_position(new_list)
if (new_notices==-1):
    print('error') 
else:
    for i in range(1,len(new_notices)+1):
        list201.append(new_notices[-i])

In [ ]:
print(list201)

## Method 3:

In [ ]:
def has_class(tag):
    return tag.has_attr('class') and tag.has_attr('colspan') 

In [ ]:
tags2 = soup.find_all(has_class)

In [ ]:
list301 = []
for each in tags2:
    list301.append(replace_punctuation(each.text))


## Method 4

In [ ]:
font_tag = soup.find('td', class_ = "list-data-focus").font
print(font_tag)
b_tag = soup.find('td', class_ ='list-data-focus').b
print(b_tag)

## Method 5

In [ ]:
all_a_tags = soup.find_all(name='a')
